In [ ]:
from __future__ import print_function, division
%pylab inline
import numpy as np
import pandas as pd
import seaborn as sns
from bokeh.plotting import *

from bokeh.io import output_notebook
output_notebook()
from collections import Counter
from operator import itemgetter

import sys
import bs4 as bs
import time
import dryscrape
import time
import json
from tqdm import tqdm
import urllib
from zipfile import ZipFile
import os
from itertools import groupby
import pysrt

from IPython.display import Image

if 'linux' in sys.platform:
    # start xvfb in case no X is running. Make sure xvfb 
    # is installed, otherwise this won't work!
    dryscrape.start_xvfb()
    
import traceback
from tqdm import tqdm, tqdm_pandas
    
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('udacity_courses.csv')
print(len(df))
df.head(2)

In [ ]:
dl_base_url = 'https://www.udacity.com/course/progress#!/c-'
df['transcripts_url'] = dl_base_url + df['id']
df.head(2)

In [ ]:
username = u'f277779@mvrht.com'
pw = u'BigDataU'

In [ ]:
def init_session(url, username, pw, wait=8):
    sess = dryscrape.Session()
    sess.set_viewport_size(width=1024, height=800)
    sess.set_attribute('auto_load_images', False)
    sess.set_attribute('javascript_can_open_windows', True)
    sess.visit(url)
    #time.sleep(wait)
    mail_field = sess.at_xpath("//input[contains(@type, 'email')]")
    if mail_field is None:
        print('already logged in!')
        render_fn = 'page.png'
        sess.render(render_fn)
    else:
        mail_field.set(username)
        pw_field = sess.at_xpath("//input[contains(@id, 'password')]")
        pw_field.set(pw)
        login_button = sess.at_xpath("//button[contains(@type, 'submit')]")
        login_button.click()
        time.sleep(wait)
    render_fn = 'page.png'
    sess.render(render_fn)
    return sess

def get_page_session(url, sess=None, wait=8):
    if sess is None:
        sess = init_session
    sess.visit(url)
    time.sleep(wait)
    #render_fn = 'page.png'
    #sess.render(render_fn)
    return sess

def download_transscript(url, sess=None, wait=8):
    course_id = url.rsplit('/')[-1][2:]
    #print('process:', course_id)
    dl_folder = 'transcripts_zips/'
    zip_fname = dl_folder + course_id + '.zip'
    if os.path.isfile(zip_fname):
        try:
            with ZipFile(zip_fname, 'r') as f:
                pass
            return zip_fname
        except:
            print(traceback.format_exc())
    if not os.path.isdir(dl_folder):
        os.makedirs(dl_folder)
    fails = 0
    while True:
        try:
            sess = get_page_session(url, sess, wait)
            transcripts_dl_button = sess.at_xpath('//div[contains(@data-supplemental-material-item, "_transcript_zip_archive")]')
            if transcripts_dl_button is None or not transcripts_dl_button.is_visible():
                print(course_id, 'no transript:', url)
                return None
            sess.exec_script('window.open = function (url, windowName, windowFeatures) { \
                                        window.dl_url = url; \
                                        return url; \
                                    }')
            transcripts_dl_button.click()
            dl_url = sess.eval_script('window.dl_url;')
            if course_id in dl_url:
                urllib.urlretrieve(dl_url, zip_fname)
                break
            else:
                print('wrong dl url.', course_id, dl_url)
                print('retry')
                fails += 1
                if fails > 10:
                    raise Exception()
        except:
            print('course:', course_id)
            print('20 sec timeout')
            render_fn = 'page.png'
            sess.render(render_fn)
            fails += 1
            if fails > 2:
                print(traceback.format_exc())
                print(course_id, 'no transript:', url)
                return None
            time.sleep(20)
            sess = init_session('https://www.udacity.com/account/auth#!/signin', username, pw)

    return zip_fname

In [ ]:
sess = init_session('https://www.udacity.com/account/auth#!/signin', username, pw)
# Image('page.png')
print('done')

In [ ]:
tqdm_pandas(tqdm(position=0))
df['transcripts_zip_fn'] = df['transcripts_url'].progress_apply(download_transscript, args=(sess, 15))

In [ ]:
Image('page.png')

In [ ]:
from collections import namedtuple
from itertools import groupby

# http://stackoverflow.com/a/23620587
def parse_srt_content(text):
    # "chunk" our input file, delimited by blank lines
    res = [list(g) for b, g in groupby(text, lambda x: bool(x.strip())) if b]

    Subtitle = namedtuple('Subtitle', 'number start end content')

    subs = []

    for sub in res:
        if len(sub) >= 3: # not strictly necessary, but better safe than sorry
            sub = [x.strip() for x in sub]
            # number, start_end, *content = sub # py3 syntax
            number = sub[0]
            start_end = sub[1]
            content = sub[2:]
            #print(start_end)
            start, end = start_end.split(' --> ')
            subs.append(Subtitle(number, start, end, content))
    return subs

In [ ]:

def extract_transripts_text(fn):
    if fn is None:
        return ''
    text = list()
    with ZipFile(fn, 'r') as zf:
        files = [item for item in zf.namelist() if item.endswith('.srt')]
        for srt_f in files:
            with zf.open(srt_f) as f:
                subs = parse_srt_content(f)
                for s in subs:
                    text.extend(s.content)
    text = ' '.join(text)
    text = text.replace('>>', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    return text

In [ ]:
df['transcript'] = df['transcripts_zip_fn'].apply(extract_transripts_text)

In [ ]:
for i in df['transcript'].head(20).values:
    print(i)

In [ ]:
df.to_pickle('udacity_courses.df')

In [ ]:
df.columns

In [ ]:
df[['id', 'transcript']].head(2)

In [ ]:
txt_folder = 'transcripts_txt/'
if not os.path.isdir(txt_folder):
    os.makedirs(txt_folder)
for idx, (c_id, trans) in df[['id', 'transcript']].iterrows():
    with open(txt_folder + c_id + '.txt', 'w') as f:
        f.write(trans)